# Library

In [ ]:
!pip install trl wandb

In [172]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from datasets import load_dataset, Dataset
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

from trl.core import LengthSampler
from trl import AutoModelForCausalLMWithValueHead
from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
)

# Reinforcement Learning with Human Feedback

paper: https://arxiv.org/pdf/2203.02155

![](https://huyenchip.com/assets/pics/rlhf/6-sft-rlhf.png)

<br>

<font style="font-size:20px"> 기존 모델의 한계 </font>

**신뢰할 수 없는 답변이나, 유해하거나, 유저에게 도움이 안 되는 결과를 생성**

## RLHF

사람의 선호를 reward로 하여 모델을 fine-tuning

<br>

<font style="font-size:20px"> 학습 방법 </font>

1. 레이블러는 입력 프롬프트에서 원하는 동작의 데모 제공. <br>
지도 학습을 사용하여 이 데이터에서 사전 학습된 GPT-3 모델 fine-tuning.

2. 주어진 입력에서 레이블러가 선호하는 output 사이의 비교 데이터 수집. <br>
reward model을 학습시켜 선호하는 출력 예측

3. PPO를 사용하여 reward model에 대한 policy 최적화.

### Model

1. Supervised Fine-Tuning (SFT): <br>
레이블된 데이터로 GPT-3 모델을 미세 조정

2. Reward Modeling (RM): <br>
SFT 모델에서 prompt와 response를 기반으로 reward를 출력하도록 모델 학습
방법:

3. Reinforcement Learning (RL): <br>
PPO를 사통해 SFT 모델을 미세 조정하는 것.

## Practice

## 긍정 답변 유도

In [6]:
# GPT-2 모델용 사전 학습된 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained('lvwerra/gpt2-imdb')

# 패딩 토큰을 설정합니다. 여기서는 EOS(End Of Sentence) 토큰을 패딩 토큰으로 지정합니다.
tokenizer.pad_token = tokenizer.eos_token

# IMDb 데이터셋의 'train' split을 로드합니다.
data = load_dataset('imdb', split='train')

# 데이터셋의 'text' 열 이름을 'review'로 변경하여 일관성을 유지합니다.
data = data.rename_columns({'text': 'review'})

# 각 리뷰 텍스트 길이가 200자 이상인 샘플만 필터링합니다.
data = data.filter(lambda x: len(x['review']) > 200)

# 데이터셋의 텐서 형식을 'pt' (PyTorch 텐서)로 설정하여 모델에 바로 사용할 수 있도록 합니다.
data.set_format('pt')

In [23]:
# 입력 텍스트의 최소 및 최대 길이를 정의합니다.
input_min_text_length = 2    # 최소 텍스트 길이
input_max_text_length = 8    # 최대 텍스트 길이

# LengthSampler 객체를 사용하여, 입력 텍스트 길이를 최소와 최대 길이 범위 내에서 무작위로 샘플링합니다.
input_size = LengthSampler(input_min_text_length, input_max_text_length)

In [83]:
# IMDb 데이터셋의 각 샘플에 대해 토큰화 및 변환을 수행하는 함수입니다.
def tokenize(sample):
    # 'review' 텍스트를 토크나이즈하고, 지정된 입력 길이(input_size)로 자릅니다.
    sample['input_ids'] = tokenizer.encode(sample['review'])[: input_size()]
    
    # 토큰화된 'input_ids'를 다시 텍스트로 디코딩하여 'query' 키에 저장합니다.
    sample['query'] = tokenizer.decode(sample['input_ids'])
    
    # 토큰화가 완료된 샘플을 반환합니다.
    return sample

In [85]:
# IMDb 데이터셋의 각 샘플에 대해 'tokenize' 함수를 적용하여 텍스트를 토큰화하고 필요한 필드를 추가합니다.
data = data.map(tokenize)

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [91]:
# 데이터 로더의 배치 단위로 데이터를 구성하는 collator 함수입니다.
def collator(data):
    # 데이터 리스트(data)에서 동일한 키의 값을 모아 배치를 구성하는 딕셔너리를 반환합니다.
    # 각 키에 대해 해당 키를 가진 모든 샘플의 값을 리스트로 묶습니다.
    return dict((key, [datum[key] for datum in data]) for key in data[0])

In [88]:
# 'lvwerra/gpt2-imdb'로 사전 학습된 Causal Language Model(자연어 생성 모델)을 로드합니다.
# 강화 학습(RL)을 위해 value head를 추가하여 감정이나 보상 등 특정 값 예측이 가능하도록 합니다.
model = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

# 학습 전 원래 사전 학습된 모델을 참조 모델로 로드합니다.
# 이는 학습 후 모델이 원래 모델과 얼마나 다른지 비교하거나 평가할 때 사용됩니다.
model_reference = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [131]:
# Proximal Policy Optimization(PPO) 구성 설정을 정의합니다.
config = PPOConfig(
    model_name='lvwerra/gpt2-imdb',  # 사용할 모델 이름
    learning_rate=2e-5,              # 학습률
    log_with='wandb',                # 실험 추적 도구로 Weights and Biases를 사용하도록 지정
)

# PPO 알고리즘을 사용하여 모델을 학습시키기 위한 트레이너(PPOTrainer)를 초기화합니다.
ppo_trainer = PPOTrainer(
    config,               # PPO 구성 설정
    model,                # 학습할 모델
    model_reference,      # 참조 모델 (기존 사전 학습된 모델)
    tokenizer,            # 토크나이저
    dataset=data,         # 데이터셋 (IMDb 데이터셋을 전처리한 것)
    data_collator=collator,  # 배치 구성을 위한 collator 함수
)

C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\trl\trainer\ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\trl\trainer\ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\accelerate\accelerator.py:443: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")


In [99]:
# Hugging Face의 파이프라인을 사용하여 감정 분석 모델을 초기화합니다.
model_reward = pipeline(
    'sentiment-analysis',  # 감정 분석 태스크 지정
    model='lvwerra/distilbert-imdb',  # 사용할 사전 학습된 DistilBERT 모델 지정
)

In [141]:
# 'bert-base-uncased'로 사전 학습된 BERT 모델을 로드하여 텍스트 분류 작업을 위한 모델을 초기화합니다.
bert = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
# 'this move is horrible'라는 텍스트에 대한 감정 분석을 수행합니다.
# top_k 파라미터를 None으로 설정하여 모델이 모든 감정 클래스의 점수를 반환하도록 합니다.
result = model_reward('this move is horrible', top_k=None)

[{'label': 'NEGATIVE', 'score': 0.994786262512207},
 {'label': 'POSITIVE', 'score': 0.0052136643789708614}]

In [102]:
# 출력 텍스트의 최소 및 최대 길이를 정의합니다.
output_min_length = 4    # 최소 출력 길이
output_max_length = 20   # 최대 출력 길이

# LengthSampler 객체를 사용하여, 출력 텍스트 길이를 최소와 최대 길이 범위 내에서 무작위로 샘플링합니다.
output_length_sampler = LengthSampler(
    output_min_length,     # 출력 길이의 최소값
    output_max_length      # 출력 길이의 최대값
)

In [ ]:
# PPO 트레이너의 데이터 로더를 통해 각 에포크와 배치를 반복합니다.
for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    # 현재 배치에서 입력 쿼리의 ID를 가져옵니다.
    queries = batch.get('input_ids')

    response_tensors = []  # 생성된 응답을 저장할 리스트

    # 각 쿼리에 대해 응답을 생성합니다.
    for query in queries:
        # 출력 길이를 샘플링합니다.
        generation_length = output_length_sampler()
        
        # 모델을 사용하여 응답을 생성합니다.
        response = ppo_trainer.generate(
            query,
            min_length=-1,           # 생성할 최소 길이
            top_k=0,                 # Top-k 샘플링 비활성화
            top_p=1,                 # Top-p 샘플링을 사용하여 무작위 생성
            do_sample=True,          # 샘플링 모드 활성화
            pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 ID 설정
            max_new_tokens=generation_length  # 생성할 최대 새로운 토큰 수
        )
        
        # 응답의 길이를 계산하여 응답 텐서를 저장합니다.
        response_length = len(response) - len(query)
        response_tensors.append(response[-response_length:])
    
    # 생성된 응답을 디코딩하여 배치에 추가합니다.
    batch['response'] = [tokenizer.decode(response.squeeze())
                         for response in response_tensors]
    
    # 쿼리와 응답을 합쳐서 텍스트 리스트를 생성합니다.
    texts = [''.join([query, response])
             for query, response
             in zip(batch.get('query'), batch.get('response'))]
    
    # 합쳐진 텍스트에 대한 감정 점수를 계산합니다.
    sentiment_scores = model_reward(
        texts,
        top_k=None,              # 모든 감정 클래스의 점수를 반환
        batch_size=32,          # 배치 크기 설정
    )
    
    # 긍정 점수를 추출하여 보상으로 사용합니다.
    positive_scores = [
        item.get('score')
        for sentiment_score in sentiment_scores
        for item in sentiment_score
        if item.get('label') == 'POSITIVE'
    ]
    
    # 보상을 텐서로 변환합니다.
    rewards = [torch.tensor(score) for score in positive_scores]

    # PPO 트레이너의 한 스텝을 진행하고 통계를 기록합니다.
    stats = ppo_trainer.step(queries, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, response_tensors)

In [160]:
def make_sentiment_form(text, model, tokenizer):
    # 입력 텍스트를 토크나이저를 사용하여 텐서 형태로 변환합니다.
    tokenized = tokenizer(text, return_tensors='pt')
    
    # 모델의 그래디언트 계산을 비활성화하여 메모리 사용을 최적화합니다.
    with torch.no_grad():
        # 토큰화된 입력을 모델에 전달하여 예측 결과를 얻습니다.
        outputs = model(**tokenized)
    
    # 모델의 로짓에 소프트맥스 함수를 적용하여 확률 분포를 계산합니다.
    probs = F.softmax(outputs.logits)[0]
    
    # 긍정 및 부정의 확률 점수를 기반으로 결과를 구성합니다.
    positive = {'label': 'POSITIVE', 'score': probs[1].item()}  # 긍정 점수
    negative = {'label': 'NEGATIVE', 'score': probs[0].item()}  # 부정 점수
    
    # 긍정 및 부정 결과를 리스트 형태로 반환합니다.
    return [positive, negative]

In [161]:
# 사용할 BERT 모델과 토크나이저의 이름을 정의합니다.
model_name = 'bert-base-uncased'

# 사전 학습된 BERT 모델을 로드합니다. 이 모델은 텍스트 분류를 위한 것입니다.
bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 해당 모델에 맞는 토크나이저를 로드합니다.
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)

# 감정 분석을 수행할 입력 텍스트를 정의합니다.
text = 'how\'s your days going on?'

# 정의한 텍스트에 대해 감정 분석을 수행하고 결과를 반환합니다.
sentiment_result = make_sentiment_form(text, bert_model, bert_tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\USER\AppData\Local\Temp\ipykernel_26376\1922592531.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(outputs.logits)[0]


[{'label': 'POSITIVE', 'score': 0.40136656165122986},
 {'label': 'NEGATVE', 'score': 0.5986334085464478}]

In [ ]:
## 위의 rlhf의 코드에서 model_reward를 직접 학습한 bert model로 변경
## data: imdb
## model: AutoModelForSequenceClassification

## 단계
## - 1: imdb data로 bert 모델 학습
## BERT 모델과 토크나이저를 초기화합니다.
model_name = 'google-bert/bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## IMDb 데이터셋의 각 샘플을 전처리하는 함수입니다.
def preprocessing(sample):
    return tokenizer(
        sample['text'],
        padding='max_length',
        max_length=256,
        truncation=True,
        return_tensors='pt'
    )

## 데이터셋에 전처리 함수를 적용하고 훈련 및 검증 데이터셋을 나눕니다.
data = load_dataset('imdb')
train = data.map(preprocessing, batched=True)
eval = train.select(range(20000, 25000)) # 평가 데이터셋: 20000~25000번째 샘플
train = train.select(range(20000))       # 훈련 데이터셋: 처음 20000개의 샘플

## 모델 훈련을 위한 하이퍼파라미터를 설정합니다.
training_args = TrainingArguments(
    output_dir='./imdb',            # 모델과 로그 저장 경로
    eval_strategy='epoch',          # 매 에포크마다 평가를 수행
    learning_rate=2e-5,             # 학습률
    warmup_steps=50,                # 웜업 스텝 설정
    per_device_train_batch_size=16, # 훈련 배치 크기
    per_device_eval_batch_size=16,  # 평가 배치 크기
    num_train_epochs=2,             # 에포크 수
    weight_decay=0.01,              # 가중치 감소율
    logging_dir='./imdb',           # 로그 디렉토리
)

## Trainer를 초기화하여 BERT 모델을 IMDb 데이터셋으로 학습할 준비를 합니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,  # 훈련 데이터셋 설정
    eval_dataset=eval,    # 평가 데이터셋 설정
)

## 모델을 훈련합니다.
trainer.train()

## 훈련이 완료된 모델을 저장합니다.
model.save_pretrained('./imdb')

## - 2: 학습된 모델을 model_reward로 로드
##      - 학습된 모델을 'model_reward'로 로드하여, 이후 RLHF에서 보상 모델로 사용합니다.
model_reward = AutoModelForSequenceClassification.from_pretrained('./imdb')
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

## - 3: model의 결괏값을 아래의 형태로 변경
##      [{'label': label, 'score': score}, {'label': label, 'score': score}]
##      - positive: 긍정 예측 점수, negative: 부정 예측 점수
def make_sentiment_form(text, model, tokenizer):
    tokenized = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokenized)
    probs = F.softmax(outputs.logits)[0]
    
    positive = {'label': 'POSITIVE', 'score': probs[1].item()}
    negative = {'label': 'NEGATVE', 'score': probs[0].item()}
    
    return [positive, negative]

## 4: RLHF 단계 - 학습을 위한 데이터 토크나이즈
input_min_text_length = 2
input_max_text_length = 8
input_size = LengthSampler(input_min_text_length, input_max_text_length)

## 샘플을 토크나이즈하고 쿼리 텍스트를 저장합니다.
def tokenize(sample):
    sample['input_ids'] = tokenizer.encode(sample['review'])[: input_size()]
    sample['query'] = tokenizer.decode(sample['input_ids'])

    return sample

# 데이터셋에 tokenize 함수를 적용하여 샘플을 토크나이즈합니다.
data = data.map(tokenize)

# 데이터 배치를 위한 collator를 설정합니다. 각 키에 대해 데이터에서 해당 키의 값을 리스트로 수집합니다.
def collator(data):
    return dict((key, [datum[key] for datum in data]) for key in data[0])

# PPO 학습을 위해 모델과 참조 모델을 초기화합니다.
model = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')
model_reference = AutoModelForCausalLMWithValueHead.from_pretrained('lvwerra/gpt2-imdb')

# PPO 학습 구성 설정
config = PPOConfig(
    model_name='lvwerra/gpt2-imdb',  # 모델 이름
    learning_rate=2e-5,               # 학습률 설정
    log_with='wandb',                 # 로그 수집 방식
)

# PPOTrainer 초기화 - PPO 알고리즘을 사용하여 모델을 학습할 준비를 합니다.
ppo_trainer = PPOTrainer(
    config,
    model,
    model_reference,
    tokenizer,
    dataset=data,
    data_collator=collator,
)

# 생성 응답의 길이를 설정하는 샘플러를 초기화합니다.
output_min_length = 4
output_max_length = 20
output_length_sampler = LengthSampler(
    output_min_length,
    output_max_length,
)

# RLHF 학습 루프 - 각 에포크마다 보상을 계산하고 모델을 업데이트합니다.
for epoch, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    queries = batch.get('input_ids')  # 현재 배치의 쿼리 입력을 가져옵니다.

    # 응답 생성 및 응답 텐서를 수집합니다.
    response_tensors = []
    for query in queries:
        generation_length = output_length_sampler()  # 생성할 응답의 길이를 샘플링합니다.
        response = ppo_trainer.generate(
            query,
            min_length=-1,
            top_k=0,
            top_p=1,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,  # 패딩 토큰 설정
            max_new_tokens=generation_length  # 최대 생성 토큰 수 설정
        )
        response = response[0]  # 첫 번째 응답을 선택합니다.
        response_length = len(response) - len(query)  # 응답의 길이를 계산합니다.
        response_tensors.append(response[-response_length:])  # 응답을 수집합니다.

    # 응답을 디코딩하여 배치에 추가합니다.
    batch['response'] = [tokenizer.decode(response.squeeze())
                         for response in response_tensors]

    # 쿼리와 응답 텍스트를 합쳐 감성 분석을 수행하고 긍정 점수를 추출합니다.
    texts = [''.join([query, response])
             for query, response
             in zip(batch.get('query'), batch.get('response'))]
    sentiment_scores = [
      make_sentiment_form(text, model_reward, bert_tokenizer)
      for text in texts
    ]
    
    # 긍정 감정 점수를 추출합니다.
    positive_scores = [
        item.get('score')
        for sentiment_score in sentiment_scores
        for item in sentiment_score
        if item.get('label') == 'POSITIVE'
    ]

    # 보상을 생성하여 PPO 트레이너의 학습 단계로 전달합니다.
    rewards = [torch.tensor(score) for score in positive_scores]

    # 학습 통계를 기록하고 PPO 학습 단계를 실행합니다.
    stats = ppo_trainer.step(queries, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)  # 학습 통계와 보상을 로그합니다.

In [ ]:
# 결과 비교

num_samples = 16  # 샘플링할 데이터 개수를 16개로 설정
game_data = {}  # 결과 데이터를 저장할 딕셔너리 생성
data.set_format('pandas')  # 데이터셋 형식을 pandas DataFrame으로 설정
df_samples = data[:].sample(num_samples)  # 데이터셋에서 16개의 샘플을 랜덤으로 추출하여 DataFrame으로 저장
game_data['query'] = df_samples['query'].tolist()  # 샘플의 'query' 열을 리스트로 변환하여 game_data에 저장
queries = df_samples['input_ids'].tolist()  # 샘플의 'input_ids' 열을 리스트로 변환하여 queries에 저장
device = 'cuda'  # 모델을 'cuda' 장치에서 실행하도록 설정

responses_reference, responses = [], []  # 참고 응답과 실제 응답을 저장할 리스트 초기화

for i in range(num_samples):  # 샘플 개수만큼 반복
    query = torch.tensor([queries[i]]).to(device)  # 현재 쿼리를 텐서로 변환하여 GPU 장치로 이동

    generation_length = output_length_sampler()  # 출력할 응답 길이를 샘플링

    # 참고 모델로 응답 생성
    response_reference = model_reference.generate(
        query,
        min_length=-1,  # 최소 생성 길이를 -1로 설정
        top_k=0,  # top-k 샘플링 비활성화
        top_p=1,  # top-p 샘플링 비활성화
        do_sample=True,  # 샘플링 방식으로 응답 생성
        pad_token_id=tokenizer.eos_token_id,  # padding에 사용할 토큰 설정
        max_new_tokens=generation_length,  # 생성할 최대 토큰 길이 설정
    )[0]
    response_length = len(response_reference) - len(query)  # 생성된 응답의 실제 길이 계산
    responses_reference.append(response_reference[-response_length:])  # 참고 응답 리스트에 추가

    # 모델로 실제 응답 생성
    response = model.generate(
        query,
        min_length=-1,  # 최소 생성 길이를 -1로 설정
        top_k=0,  # top-k 샘플링 비활성화
        top_p=1,  # top-p 샘플링 비활성화
        do_sample=True,  # 샘플링 방식으로 응답 생성
        pad_token_id=tokenizer.eos_token_id,  # padding에 사용할 토큰 설정
        max_new_tokens=generation_length,  # 생성할 최대 토큰 길이 설정
    )[0]
    response_length = len(response) - len(query)  # 생성된 응답의 실제 길이 계산
    responses.append(response[-response_length:])  # 실제 응답 리스트에 추가

In [ ]:
# `responses_reference`의 각 응답을 디코딩하여 수정 전 원래 응답을 얻음.
game_data['response (before)'] = [
    tokenizer.decode(response_reference) for response_reference in responses_reference
]

# `responses`의 각 응답을 디코딩하여 수정 후 응답을 얻음.
game_data['response (after)'] = [
    tokenizer.decode(response) for response in responses
]

# 각 쿼리와 해당하는 수정 전 응답을 연결하여 텍스트 샘플 리스트를 만듦.
texts = [
    ''.join([query, response])
    for query, response
    in zip(game_data.get('query'), game_data.get('response (before)'))
]

# 수정 전 쿼리-응답 텍스트 샘플에 대해 감정 점수를 계산함.
# `top_k=None` 옵션은 모든 감정 레이블의 점수를 반환함.
sentiment_scores = model_reward(texts, top_k=None)

# 'POSITIVE' 레이블로 지정된 감정 점수를 필터링하여 추출함.
positive_scores = [
    item.get('score')
    for sentiment_score in sentiment_scores
    for item in sentiment_score
    if item.get('label') == 'POSITIVE'
]

# 수정 전 응답에 대한 긍정 감정 점수를 game_data 딕셔너리에 저장함.
game_data['rewards (before)'] = positive_scores

# 수정 후 응답에 대해서도 동일한 과정을 반복함.
# 각 쿼리와 해당하는 수정 후 응답을 연결하여 텍스트 샘플 리스트를 만듦.
texts = [
    ''.join([query, response])
    for query, response
    in zip(game_data.get('query'), game_data.get('response (after)'))
]

# 수정 후 쿼리-응답 텍스트 샘플에 대해 감정 점수를 계산함.
sentiment_scores = model_reward(texts, top_k=None)

# 'POSITIVE' 레이블로 지정된 감정 점수를 필터링하여 추출함.
positive_scores = [
    item.get('score')
    for sentiment_score in sentiment_scores
    for item in sentiment_score
    if item.get('label') == 'POSITIVE'
]

# 수정 후 응답에 대한 긍정 감정 점수를 game_data 딕셔너리에 저장함.
game_data['rewards (after)'] = positive_scores

In [ ]:
# game_data 딕셔너리를 데이터프레임으로 변환하여 결과를 표 형태로 생성함
results = pd.DataFrame(game_data)

# 데이터프레임 출력
results